In [ ]:
## * install pymilvus sentence-transformers library 
!pip install --no-cache-dir python-dotenv pymilvus sentence-transformers 

In [ ]:
from dotenv import load_dotenv
import os
load_dotenv()

## * Loading environment variables
MILVUS_PORT =os.getenv("MILVUS_PORT")
MILVUS_HOST =os.getenv("MILVUS_HOST")

In [4]:
## * bash standalone_embed.sh start
## * start Milvus standalone
from pymilvus import (
    connections,utility,FieldSchema,CollectionSchema,DataType,Collection
)
connections.connect(host=MILVUS_HOST,port=MILVUS_PORT)

In [19]:
COLLECTION_NAME = "movies_db"
DIMENSION = 384 ## sentence-transformers generate 384 embedding dimension.

## * Generally more dimensions means more information can be stored. and more accurate result
## * adding field to collection
fields = [
    FieldSchema(name='id', dtype=DataType.INT64, is_primary=True, auto_id=True), ## id is auto=increment
    FieldSchema(name='title', dtype=DataType.VARCHAR, max_length=200),
    FieldSchema(name='embedding', dtype=DataType.FLOAT_VECTOR, dim=DIMENSION)
]
schema = CollectionSchema(fields=fields)
collection = Collection(name=COLLECTION_NAME, schema=schema)


In [20]:
# * create an IVF_FLAT index for collection.
index_params = {
    'metric_type':'L2',
    'index_type':"IVF_FLAT",
    'params':{'nlist': 1536}
}
collection.create_index(field_name="embedding", index_params=index_params)
collection.load()

In [21]:
# * inserting data
import csv

def csv_load(file):
    with open(file, newline='') as f:
        reader = csv.reader(f, delimiter=',')
        for row in reader:
            if '' in (row[1], row[7]):
                continue
            yield (row[1], row[7])

In [ ]:
## * processing embeddings in batch because it takes time to insert them.
import time
import db

BATCH_SIZE = 128

data_batch = [[],[]]

count = 0

for title, plot in csv_load('plots.csv'):
    data_batch[0].append(title)
    data_batch[1].append(plot)
    if len(data_batch[0]) % BATCH_SIZE == 0:
        db.insert_data(data_batch)
        data_batch = [[],[]]
        print(f"\ninserted... {count} movies")
    count += 1

if len(data_batch[0]) != 0:
    db.insert_data(data_batch)

collection.flush()

In [ ]:
import db

search_terms = ['A movie about cars', 'A movie about monsters']

result = db.search_data(search_terms)
result